Paired TAM (transcient abnormal myleoproliferation) vs AML (acute myeloid leukemia) analysis of patients with the co-occuring condition of Down Syndrome using IJC counts obtained from rMATS analysis.

Using a matrix constructed from Kids First Workflow V4 done on single runs, a post-rMATS-single-run prepareSEfiles.sh was run that created a bed file for visualizaiton in UCSC Genome browser of all the events, as well as created a matrix of the single runs normalized to the non-redundant union of files.  Using associative arrays in an awk script, it was a rapid way to transform the individual counts from each of the individual runs into a matrix that facilitated analysis.

Using annotations obtained from the rMATS run that provided the coordinates of each of the splicing events as well as the gene that the junctions came from and the count of the reads that overlapped the junctions.   
 
We will use Limma to perform analysis of these junction counts in the identical way that a gene analysis would be completed.


In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager", repos = "https://cloud.r-project.org")


In [ ]:
setwd("../data/")


In [ ]:
getwd()


In [ ]:
cts <- as.matrix(read.csv("SE.IJC.paired.TAM.AML.csv",sep=",",row.names="ID"))

In [ ]:
featureData <- data.frame(read.csv("SE.coordinates.matrix.csv", sep=",",row.names="ID"))

In [ ]:
head(featureData,2)

In [ ]:
featureData <- featureData[,c(1,2)]

In [ ]:
head(featureData,2)

In [ ]:
dim(cts)
head(cts,2)

In [ ]:
dim(featureData)
head(featureData,2)

In [ ]:
coldata <- read.csv("design_matrix.csv",row.names=1)

In [ ]:
coldata


In [ ]:
coldata <- coldata[,c("patient","condition")]
coldata$condition <- factor(coldata$condition)
coldata$patient <- factor(coldata$patient)

In [ ]:
rownames(coldata)

In [ ]:
rownames(coldata) <-sub("-",".",rownames(coldata))

In [ ]:
colnames(cts)

In [ ]:
all(rownames(coldata) %in% colnames(cts))



In [ ]:
library(Glimma)

In [ ]:
BiocManager::install("dplyr")

In [ ]:
library(dplyr)

In [ ]:
colnames(cts)

In [ ]:
grouping_variable <- c("TAM","AML","TAM","AML","TAM","AML","TAM","AML")
grouping_variable

In [ ]:
TAM_group <- cts[,grouping_variable == "TAM"]
colnames(TAM_group)
TAM_group_df <- data.frame(TAM_group)
AML_group <- cts[,grouping_variable == "AML"]
AML_group_df <- data.frame(AML_group)
colnames(AML_group)

In [ ]:
dim(cts)
head(cts,4)

In [ ]:
TAM_rowmeans <- rowMeans(TAM_group_df,na.rm=TRUE)
head(TAM_rowmeans,3)
length(TAM_rowmeans)
AML_rowmeans <- rowMeans(AML_group_df,na.rm=TRUE)
head(AML_rowmeans,3)
length(AML_rowmeans)

In [ ]:
install.packages("matrixStats")
library(matrixStats)

In [ ]:
TAM_rowsds = rowSds(as.matrix(TAM_group_df))
AML_rowsds = rowSds(as.matrix(AML_group_df))
length(TAM_rowsds)
length(AML_rowsds)

In [ ]:
TAM_withinsds <- as.logical(abs((TAM_group[,1] - TAM_rowmeans) <= TAM_rowsds)) &
                 as.logical(abs((TAM_group[,2] - TAM_rowmeans) <= TAM_rowsds)) &
                 as.logical(abs((TAM_group[,3] - TAM_rowmeans) <= TAM_rowsds)) &
                 as.logical(abs((TAM_group[,4] - TAM_rowmeans) <= TAM_rowsds)) 
is.logical(TAM_withinsds)
length(TAM_withinsds)
dim(TAM_withinsds)
head(TAM_withinsds)
sum(TAM_withinsds == TRUE)

In [ ]:
AML_withinsds <- as.logical(abs((AML_group[,1] - AML_rowmeans) <= AML_rowsds)) &
                 as.logical(abs((AML_group[,2] - AML_rowmeans) <= AML_rowsds)) &
                 as.logical(abs((AML_group[,3] - AML_rowmeans) <= AML_rowsds)) &
                 as.logical(abs((AML_group[,4] - AML_rowmeans) <= AML_rowsds))
is.logical(AML_withinsds)
length(AML_withinsds)
dim(AML_withinsds)
head(AML_withinsds)
sum(AML_withinsds == TRUE)

In [ ]:
filter_cts_logical <- AML_withinsds & TAM_withinsds
is.logical(filter_cts_logical)
length(filter_cts_logical)
dim(filter_cts_logical)
head(filter_cts_logical)
sum(filter_cts_logical == TRUE)

In [ ]:
head(cts,2)
filtered_cts <- cts[filter_cts_logical,]
dim(filtered_cts)

now comes the question of ratios.  To permit the comparison and analysis of the group as distinguished by difference between the two conditions, TAM and AML, we do analysis on the ratios rather than allowing statistical analsys between the two groups.   What we are looking for are the signals that indicate what would be important.  This is general characterization analysis rather than differential analysis.  

In [ ]:
# lets look at limma/voom
BiocManager::install("limma")

In [ ]:
BiocManager::install("statmod")

In [ ]:
library(limma)
library(edgeR)
library(statmod)

In [ ]:
# making a counts matrix
dge <- DGEList(counts=filtered_cts)

In [ ]:
colnames(dge)

In [ ]:
head(dge,2)

In [ ]:
design <- model.matrix(~ 0 + factor(c(1,2,1,2,1,2,1,2)))

In [ ]:
colnames(design) = c("TAM","AML")

In [ ]:
head(design)

In [ ]:
# normalize and filter
keep <- filterByExpr(dge, design)
dge <- dge[keep,,keep.lib.size=FALSE]

In [ ]:
# apply scale normalization
dge <- calcNormFactors(dge)

In [ ]:
# differential expression:  limma-trend
logCPM <- cpm(dge, log=TRUE, prior.count=3)

In [ ]:
fit <- lmFit(logCPM, design)

In [ ]:
fit <- eBayes(fit, trend=TRUE)

In [ ]:
topTable(fit, coef=ncol(design))

In [ ]:
# to give more weight to fold-changes in the junction ranking
fit <- lmFit(logCPM, design)
fit <- treat(fit, lfc=log2(1.2), trend=TRUE)
topTreat(fit, coef=ncol(design))

In [ ]:
# find the annotations for these junctions
lookup = rownames(topTreat(fit, coef=ncol(design)))
lookup

In [ ]:
featureData[lookup,]

In [ ]:
geneSymbol <- featureData[lookup,2]
as.matrix(geneSymbol)

In [ ]:
v <- voom(dge, design, plot=TRUE)

In [ ]:
summary(v)

In [ ]:
vfit <- lmFit(v, design)
vfit <- eBayes (vfit)
vfit <- treat(vfit, lfc=log2(1.2))
topTreat(vfit, coef=ncol(design))

In [ ]:
de_results <- topTreat(vfit, coef=ncol(design), n=Inf, sort.by="logFC")

In [ ]:
head(de_results)

In [ ]:
# Assuming you have the 'de_results' object from topTable
fold_change_threshold <- 5
adjusted_pvalue_threshold <- 0.05

# Select genes that meet both fold change and adjusted p-value criteria
significant_genes <- de_results[
  abs(de_results$logFC) > fold_change_threshold &
  de_results$adj.P.Val < adjusted_pvalue_threshold,
]


In [ ]:
dim(significant_genes)

In [ ]:
transformed_expression <- v$E

In [ ]:
head(rownames(transformed_expression))
dim(transformed_expression)

In [ ]:
significant_transformed_expression <- transformed_expression[rownames(significant_genes),]

In [ ]:
head(significant_transformed_expression,3)

In [ ]:
string_gene_list <- as.matrix(featureData[rownames(significant_genes),2])
length(string_gene_list)
#string_gene_list

In [ ]:
top_genes_expression <- dge[rownames(significant_genes),]

In [ ]:
dim(top_genes_expression)

In [ ]:
head(top_genes_expression,10)

In [ ]:
top_transformed_expression <- v$E[rownames(significant_genes),]
dim(top_transformed_expression)
rnames <- as.data.frame(featureData[rownames(significant_genes),2])
#rnames

In [ ]:
library("pheatmap")
df <- as.data.frame(coldata[,c("condition","patient")])

In [ ]:
# fc 5 - 2132 genes
pheatmap(top_genes_expression, cluster_rows5=TRUE, show_rownames=FALSE,
         cluster_cols=TRUE, annotation_col=df, scale="row")

In [ ]:
# fc 8
# Assuming you have the 'de_results' object from topTable
fold_change_threshold <- 9.5
adjusted_pvalue_threshold <- 0.05

# Select genes that meet both fold change and adjusted p-value criteria
significant_genes <- de_results[
  abs(de_results$logFC) > fold_change_threshold &
  de_results$adj.P.Val < adjusted_pvalue_threshold,
]
dim(significant_genes)

In [ ]:
top_genes_expression <- dge[rownames(significant_genes),]
# fc 5 - 2132 genes
pheatmap(top_genes_expression, cluster_rows5=TRUE, show_rownames=FALSE,
         cluster_cols=TRUE, annotation_col=df, scale="row")

In [ ]:
# find the annotations for these junctions
lookup = rownames(significant_genes)

as.matrix(featureData[lookup,2], )

In [ ]:
 results <- decideTests(vfit)

In [ ]:
dim(results)
summary(results)

In [ ]:
vennDiagram(results)

In [ ]:
v <- voom(dge, design, plot=TRUE)

In [ ]:
fit <- lmFit(v, design)

In [ ]:
fit2 <- contrasts.fit(fit,contrast.matrix)

In [ ]:
fit2 <- eBayes(fit2)

In [ ]:
topTable(fit2, adjust="BH")

In [ ]:
de_results_fit2 <- topTable(fit2, adjust="BH", n=Inf)
dim(de_results_fit2)

In [ ]:
head(de_results_fit2[,c("TAM")])


In [ ]:
de_results_fit2_ratio <- matrix(1.0,
              nrow = dim(de_results_fit2)[1],
              ncol = 1)
dim(de_results_fit2_ratio)
head(de_results_fit2_ratio,3)

In [ ]:
i = 2000

In [ ]:
de_results_fit2[i,c("TAM")] > de_results_fit2[i,c("AML")]

In [ ]:
de_results_fit2_ratio[i] = as.double((de_results_fit2[i,c("TAM")])/(de_results_fit2[i,c("AML")]))


In [ ]:
de_results_fit2_ratio[i]

In [ ]:
for (i in 1:10) {
    if (de_results_fit2[i,c("TAM")] >= de_results_fit2[i,c("AML")] ){
        de_results_fit2_ratio[i] = as.double((de_results_fit2[i,c("TAM")])/(de_results_fit2[i,c("AML")]))
    } else {
        de_results_fit2_ratio[i] = as.double((de_results_fit2[i,c("AML")])/(de_results_fit2[i,c("TAM")]))
    }
}


In [ ]:
head (de_results_fit2_ratio)

In [ ]:
results2 <- decideTests(fit2)

In [ ]:
vennDiagram(results2)

In [ ]:
vennCounts(results2)

In [ ]:
contrasts_fit_venn_counts <- vennCounts(results2)

In [ ]:
contrasts_fit_venn_counts

In [ ]:
head(results2)

In [ ]:
head(de_results2)

In [ ]:
filtered_de_results2 <- de_results2[c((abs(de_results2[,c("AMLvsTAM")]) > 1.5) |
                        (abs(de_results2[,c("TAMvsAML")]) > 1.5)),]

In [ ]:
filtered_de_results2

In [ ]:
# Assuming you have the 'de_results' object from topTable
fold_change_threshold <- 1.5
adjusted_pvalue_threshold <- 0.05

# Select genes that meet both fold change and adjusted p-value criteria
significant_genes <- de_results2[
  abs(de_results2$logFC) > fold_change_threshold &
  de_results2$adj.P.Val < adjusted_pvalue_threshold,
]


In [ ]:
summary(results2)

In [ ]:
dim(results2)

In [ ]:
library("pheatmap")
select <- 
df <- as.data.frame(colData(dds)[,c("condition","patient")])


In [ ]:
ddsLRT <- DESeq(dds, test="LRT", reduced=~1)
resLRT <- results(ddsLRT)

In [ ]:
vsd <- vst(dds, blind=FALSE)


In [ ]:
??vsd

In [ ]:
head(assay(vsd), 3)


In [ ]:
library("vsn")
meanSdPlot(assay(vsd))

In [ ]:
vsdBlindTRUE <- vst(dds, blind=TRUE)
meanSdPlot(assay(vsdBlindTRUE))

In [ ]:
# this gives log2(n + 1)
ntd <- normTransform(dds)
meanSdPlot(assay(ntd))

In [ ]:
dim(dds)

In [ ]:
library("pheatmap")
select <- order(rowMeans(counts(dds,normalized=FALSE)),
                decreasing=FALSE)[87000:97000]
df <- as.data.frame(colData(dds)[,c("condition","patient")])


In [ ]:
pheatmap(assay(ntd)[select,], cluster_rows5=FALSE, show_rownames=FALSE,
         cluster_cols=FALSE, annotation_col=df)

Heatmap of the sample-to-sample distances
Another use of the transformed data is sample clustering. Here, we apply the dist function to the transpose of the transformed count matrix to get sample-to-sample distances.



In [ ]:
sampleDists <- dist(t(assay(vsd)))


In [ ]:
library("RColorBrewer")
sampleDistMatrix <- as.matrix(sampleDists)


In [ ]:
rownames(sampleDistMatrix) <- paste(vsd$condition, vsd$patient, sep="-")
colnames(sampleDistMatrix) <- NULL
colors <- colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)


In [ ]:
pheatmap(sampleDistMatrix,
         clustering_distance_rows=sampleDists,
         clustering_distance_cols=sampleDists,
         col=colors)

In [ ]:
plotPCA(vsd, intgroup=c("condition", "patient"))


In [ ]:
library("ggplot2")

In [ ]:
pcaData <- plotPCA(vsd, intgroup=c("condition", "patient"), returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=condition, shape=patient)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
  coord_fixed()


Interactions

In [ ]:
dds$group <- factor(paste0(dds$patient, dds$condition))


In [ ]:
design(dds) <- ~ group


In [ ]:
dds <- DESeq(dds)


In [ ]:
levels(ddsMF$condition)

In [ ]:
design(ddsMF) <- formula(~ condition)


In [ ]:
ddsMF <- DESeq(ddsMF)


In [ ]:
resMFType <- results(ddsMF)
head(resMFType)

In [ ]:
plotMA(resMFType, ylim = c(-20, 20))


In [ ]:
library("BiocParallel")
register(MulticoreParam(2))

In [ ]:
plotMA(res.noshr, ylim = c(-20, 20))


In [ ]:
reslfc <- lfcShrink(dds, coef="condition_AML_vs_TAM", type="apeglm")

res here is using shrinkage to alter the variation sensitive to each of the junctions.

In [ ]:
plotMA(reslfs, ylim = c(-2, 2))

In [ ]:
resOrdered <- res[order(res$pvalue),]


In [ ]:
par(mar=c(8,5,2,2))
boxplot(log10(assays(dds)[["cooks"]]), range=0, las=2)

In [ ]:
summary(res)


In [ ]:
sum(res$padj < 0.05, na.rm=TRUE)


In [ ]:
write.csv(as.data.frame(resOrdered), 
          file="condition_AML_vs_TAM_results_apeglm_shrinkage.csv")

In [ ]:
# because we are interested in treated vs untreated, we set 'coef=2'
resNorm <- lfcShrink(dds, coef=2, type="normal")
resAsh <- lfcShrink(dds, coef=2, type="ashr")

In [ ]:
plotMA(res, ylim=c(-2,2))


In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("vsn")

In [ ]:
# this gives log2(n + 1)
ntd <- normTransform(dds)
library("vsn")
meanSdPlot(assay(ntd))

In [ ]:
meanSdPlot(assay(vsd))
